# Modulo 35 - Pratice

Com base na tarefa realizada no módulo anterior, vamos fazer um diagnóstico do modelo.

Carregue a base ```previsao_de_renda2.csv```. Separe em uma base de treino e uma base de teste.

Vamos resgatar a melhor versão do modelod e previsão de renda que você fez para esta base. 

- Substitua missings pela média
- Rode novamente o modelo na base de treino (ou desenvolva-o caso não tenha o registro guardado). 

Este modelo deve prever a variável ```renda``` com base nas demais variáveis exceto ```data_ref``` e ```index```. Já vimos que a variável renda é melhor modelada com a transformação ```log()```, não se esqueça disso.

---

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns

import scipy.stats as ss 
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

import patsy
import statsmodels.api as sm

from scipy.interpolate import interp1d
import warnings;   warnings.filterwarnings("ignore")

df = pd.read_csv('previsao_de_renda2.csv', index_col=0)
df.head()

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [5]:
# Identificando valores missing
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usage: 1

In [6]:
df.isnull().sum()

data_ref                    0
id_cliente                  0
sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [7]:
# Substituindo os valores missing pela média

df['tempo_emprego'].fillna(value=df['tempo_emprego'].mean(), inplace=True)

In [8]:
df.isnull().sum()

data_ref                 0
id_cliente               0
sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

In [13]:
# Separando bases de treino e teste baseado na data

df_train = df['2015-12-01']
df_test = df['2016-01-01':]

# Criando as variáveis log_renda

df_train['log_renda']=np.log(df_train['renda'])
df_test['log_renda']=np.log(df_test['renda'])

KeyError: '2015-12-01'

In [ ]:
# No módulo passado já obtivemos um modelo com 4 variáveis explicativas através da regressão LASSO 

X1 = patsy.dmatrices('''log_renda ~ C(posse_de_imovel)
                    + qtd_filhos 
                    + idade
                    + tempo_emprego
                    + 1''', df_train)

res = smf.ols(X1, data=df_train).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.05)
res.summary()

In [ ]:
# Aparentemente podemos até retirar mais uma variável do modelo pois o R² não se alterou

# Irei remover a variável idade também, pois representa um coeficiente muito pequeno comparado com as outras variáveis

X1 = patsy.dmatrices('''log_renda ~ C(posse_de_imovel)
                    + tempo_emprego
                    + 1''', df_train)

res = smf.ols(X1, data=df_train).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0.05)
res.summary()

---

## Verifique as suposições do modelo

Faça uma análise para avaliar as suposições básicas:

- Os resíduos desse modelo possuem distribuição Normal (ou algo próximo)?
- Verifique a independência dos resíduos
    - Faça um gráfico dos resíduos versus os valores preditos
    - Avalie se há padrões dos resíduos versus cada uma das variáveis do modelo
    - Avalie se o valor médio dos resíduos aparenta ter relação com o mês de referência
- Avalie se a variância da variável resposta parece ser conforme os valores previstos.

Você considera que as suposições estão atendidas? Há algum impacto em eventuais desvios?

In [ ]:
# Vamos analisar os resíduos

sns.displot(res.resid, height=3, aspect=2);

In [ ]:
sns.residplot(x=res.predict(), y='log_renda', data=df_train)

In [ ]:
sns.scatterplot(x = res.predict(), y = 'posse_de_imovel', data=df_train);

In [ ]:
sns.scatterplot(x = res.predict(), y = 'tempo_emprego', data=df_train);

In [ ]:
sns.scatterplot(x = res.predict(), y = 'data_ref', data=df_train);

In [ ]:
sns.scatterplot(x = res.predict(), y = res.resid)

---

## Outliers

Avalie os *studentized residuals*, verifique se há pontos que parecem ser discrepantes.

Avalie se há pontos influentes.

In [ ]:

df_train_short = df_train[:'2015-02-01']
df_train_short.reset_index(inplace=True)
df_train_short.info()

In [ ]:
reg = smf.ols('log_renda ~ tempo_emprego', data=df_train_short).fit()

In [ ]:
%%time

fig = sm.graphics.influence_plot(reg, criterion="cooks")
fig.show()

---

## Multicolinearidade

Avalie se há questões relacionadas a multicolinearidade através de pelo menos:

- Matriz de correlação de Spearman
- VIF

In [ ]:
df_train_cut = df_train[['posse_de_imovel', 'tempo_emprego', 'log_renda']]

df_train_cut.head(2)

In [ ]:
# Matriz de correlação de Spearman

df_train_cut.corr(method='spearman')


In [ ]:
# VIF

variaveis = ['tempo_emprego', 'posse_de_imovel']

dum = pd.get_dummies(df_train_cut[variaveis], drop_first=True)
dum.head(2)

In [ ]:
vif(dum.values, 1)

In [ ]:
vars_vif = pd.DataFrame()
vars_vif["VIF Factor"] = [vif(dum, i) for i in range(dum.shape[1])]
vars_vif["Feature"] = dum.columns

vars_vif.round(2)

---

## Ajustes

Faça os ajustes que julgar necessários no modelo e compare as métricas de desempenho do modelo original e ajustado na base de testes.

### Sem edições necessárias